# 模型训练

In [5]:
#  导入数据
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# 读取前1000行
# train_data = pd.read_csv('train_all.csv', nrows=1000)
# test_data = pd.read_csv('test_all.csv', nrows=1000)

# 读取全部数据
train_data = pd.read_csv('train_all.csv', nrows=None)
test_data = pd.read_csv('test_all.csv', nrows=None)

# 训练数据和测试数据处理
feature_columns = [col for col in train_data.columns if col not in ['user_id', 'label']]
train = train_data[feature_columns].values
test = test_data[feature_columns].values
target = train_data['label'].values


# folds = 5
# kf = KFold(n_splits=folds, shuffle=True, random_state=2021)

# for idx, (train_index, test_index) for enumerate(kf.split(ttain)):
#     train_x = train[train_index]
#     train_y = target[train_index]
#     test_x = test[test_index]
#     test_y = target[test_index]


# 切分数据
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.4, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_train.shape)

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)


(156518, 230) (156518,)
(104346, 230) (156518,)


0.9387710118260403

In [2]:
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier?

In [6]:
# 简单验证
from sklearn.model_selection import cross_val_score

# clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
# clf = clf.fit(X_train, y_train)

scores = cross_val_score(clf, train, target, cv=5)
print(scores)
print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

[0.93885726 0.93885726 0.9388381  0.9388381  0.93885609]
Accuracy: 0.94 (+/- 0.00)


In [8]:
# F1值进行评价
"""
'micro':Calculate metrics globally by counting the total true positives, false negatives and false positives.
'micro':通过先计算总体的TP，FN和FP的数量，再计算F1
'macro':Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
'macro':分布计算每个类别的F1，然后做平均（各类别F1的权重相同）
"""

scores = cross_val_score(clf, train, target, cv=5, scoring='f1_macro')
print(scores)
print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

[0.48423227 0.48423227 0.48422718 0.48422718 0.48423196]
Accuracy: 0.48 (+/- 0.00)


In [10]:
# 交叉验证
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(clf, train, target, cv=cv)

array([0.93870432, 0.93828265, 0.93956044, 0.94040378, 0.93763097])

In [15]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
for k, (train_index, test_index) in enumerate(kf.split(train)):
    x_train, x_test, y_train, y_test = train[train_index], train[test_index], target[train_index], target[test_index]
    clf = clf.fit(x_train, y_train)
    print(k, clf.score(x_test, y_test))
                                                                                                          

0 0.9395089414064746
1 0.9386272593103713
2 0.9374005711766623
3 0.9398539474440802
4 0.9388560913900177


In [ ]:
# 分组切分数据
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold()